# Selection algorithms
In computer science, a selection algorithm is an algorithm for finding the
**k-th smallest** value in a collection of ordered values, such as numbers.
Here we analyse quickselect and the median of medians algorithm.

## Authors
* Christian Rossi (10736464)
* Kirolos Shroubim (10719510)
* Antonio Sulfaro (10742266)



## Reference



> Introduction to algorithms \\
 Authors: Thomas H. Cormen, Charles E. Leiserson, Ronald L. Rivest, and Clifford Stein.



## Environment setup

Download and install Cmake, Google benchmark and Google test.

In [1]:
# C language import
!apt-get install -y clang
!apt-get install -y build-essential
!apt-get install -y cmake

# Google benchmark setup
!git clone https://github.com/google/benchmark.git
!git clone https://github.com/google/googletest.git benchmark/googletest
!rm -rf benchmark/build
!cmake -E make_directory "benchmark/build"
!cmake -E chdir "benchmark/build" cmake -DCMAKE_BUILD_TYPE=Release ..
!cmake --build "benchmark/build" --config Release --target install

# Google test setup
!rm -rf benchmark/googletest/build
!cmake -E make_directory "benchmark/googletest/build"
!cmake -E chdir "benchmark/googletest/build" cmake -DCMAKE_BUILD_TYPE=Release ..
!cmake --build "benchmark/googletest/build" --config Release --target install

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
clang is already the newest version (1:14.0-55~exp2).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Cloning into 'benchmark'...
remote: Enumerating objects: 8850, done.
remote: Counting objects: 100% (1450/1450), done.
remote: Compressing objects: 100% (237/237), done.
remote: Total 8850 (delta 1297), reused 1276 (delta 1199), pack-reused 7400 (from 1)
Receiving objects: 100% (8850/8850), 2.80 MiB | 11.65 MiB/s, done.
Resolving deltas:


## Algorithms implementation

Quickselect (getIthElementRand) and Median of Medians (getIthElement) implementation in C.

### Quickselect
Quickselect is an efficient selection algorithm that operates in average linear time, leveraging the principles of the quicksort algorithm to partition an array and recursively narrow down the search space for the k-th smallest element.
Its performance can degrade to quadratic time in the worst-case scenario, particularly when the input array is nearly sorted or when poor pivot choices are made.

### Median of median
The Median of Medians algorithm serves as a more robust alternative, providing a guaranteed linear time complexity in the worst case.
This approach divides the array into subarrays, recursively finds the medians of these subarrays, and uses the median of these medians as a pivot.
This ensures a more balanced partitioning and mitigates the risk of poor performance associated with traditional quickselect implementations.
In contrast is worst in the real life situation, having a worse, but still linear, average time complexity.

In [2]:
%%writefile ith_element.c

#include <stdio.h>
#include <tgmath.h>

int pivot(int* list, int left, int right);
int select (int* list, int left, int right, int i);
int rand_select(int* list, int left, int right, const int i);
int partition (int* list, int left, int right, int pivotIndex, int i);
int partition5(int* list, int left, int right);
void swap(int* list, int i1, int i2);

int getIthElement(int *list, int length, int i) {
    const int index = select(list, 0, length - 1, i);
    return list[index];
}

int getIthElementRand(int *list, int length, int i) {
    srand(time(NULL));
    const int index = rand_select(list, 0, length - 1, i);
    return index;
}

void swap(int* list, const int i1, const int i2) {
    if (list == NULL) return;
    const int temp = list[i1];
    list[i1] = list[i2];
    list[i2] = temp;
}

int pivot(int* list, int left, const int right) {
    if (right - left < 5) return partition5(list, left, right);
    for(int i = left; i <= right; i = i + 5) {
        int subRight = i + 4;
        if (subRight > right) subRight = right;
        const int median5 = partition5(list, i, subRight);

        swap(list, median5, left + (int) floor((double) (i - left) / 5));
    }
    return select(list, left, left + (int) floor((double) (right - left) / 5), (int) floor((double) (right - left) / 10) + left + 1);
}

int select(int* list, int left, int right, const int i) {
    while (1) {
        if (left == right) return left;
        int pivotIndex = pivot(list, left, right);
        pivotIndex = partition(list, left, right, pivotIndex, i);

        if (i == pivotIndex) return i;
        if (i < pivotIndex) right = pivotIndex - 1;
        else left = pivotIndex + 1;
    }
}

int rand_select(int* list, int left, int right, const int i) {
    if (left == right) return list[left];
    int pivotIndex = left + floor(rand()%(right - left + 1));
    pivotIndex = partition(list, left, right, pivotIndex, i);

    if (i == pivotIndex) return list[i];
    if (i < pivotIndex) return rand_select(list, left, pivotIndex - 1, i);
    return rand_select(list, pivotIndex + 1, right, i);
}

int partition(int* list, const int left, const int right, const int pivotIndex, const int i) {
    if (list == NULL) return -1;
    const int pivotValue = list[pivotIndex];

    swap(list, pivotIndex, right);
    int storeIndex = left;

    for (int j = left; j < right; j++) {
        if (list[j] < pivotValue) {
            swap(list, storeIndex, j);
            storeIndex++;
        }
    }

    int storeIndexEq = storeIndex;
    for(int j = storeIndex; j < right; j++) {
        if (list[j] == pivotValue) {
            swap(list, storeIndexEq, j);
            storeIndexEq++;
        }
    }

    swap(list, right, storeIndexEq);

    if (i < storeIndex) return storeIndex;
    if (i <= storeIndexEq) return i;
    return storeIndexEq;
}

int partition5(int* list, const int left, const int right) {
    int i = left + 1;
    while (i <= right) {
        int j = i;
        while (j > left && list != NULL && list[j-1] > list[j]) {
            swap(list, j-1, j);
            j--;
        }
        i++;
    }
    return left + (right - left) / 2;
}

Writing ith_element.c


This header exposes only the necessary function signatures for external use, while utility functions and implementation details are kept hidden to tests and benchmark

In [3]:
%%writefile ith_element.h

int getIthElement(int *list, int length, int i);
int getIthElementRand(int *list, int length, int i);

Writing ith_element.h


## Test



In [4]:
%%writefile test.cpp

#include "gtest/gtest.h"
#include "ith_element.h"

// test for base cases

// test case for the first element of an ordered array of negative numbers
void Test1(std::function<int(int*, int, int)> func){
  int list[] = {-1, -2, -3, -4, -5, -6, -7, -8, -9, -10};
    int length = sizeof(list)/sizeof(int);
    int i = 0;
    const int result = func(list, length, i);
    EXPECT_EQ(result, -10);
}
TEST(RandomTest, NegativeInverseOrderedFirstElement) {
    Test1([](int* list, int length, int i) {
        return getIthElement(list, length, i);
    });
}
TEST(RandomTest, NegativeInverseOrderedFirstElementRand) {
    Test1([](int* list, int length, int i) {
        return getIthElementRand(list, length, i);
    });
}

// test case for the first element of an ordered array
void Test2(std::function<int(int*, int, int)> func){
  int list[] = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10};
    int length = sizeof(list)/sizeof(int);
    int i = 0;
    const int result = func(list, length, i);
    EXPECT_EQ(result, 1);
}

TEST(RandomTest, OrderedFirstElement) {
    Test2([](int* list, int length, int i) {
        return getIthElement(list, length, i);
    });
}
TEST(RandomTest, OrderedFirstElementRand) {
    Test2([](int* list, int length, int i) {
        return getIthElementRand(list, length, i);
    });
}

// test case for the last element of an ordered array
void Test3(std::function<int(int*, int, int)> func) {
    int list[] = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10};
    int length = sizeof(list)/sizeof(int);
    int i = 9;
    const int result = func(list, length, i);
    EXPECT_EQ(result, 10);
}

TEST(RandomTest, OrderedLastElement) {
    Test3([](int* list, int length, int i) {
        return getIthElement(list, length, i);
    });
}
TEST(RandomTest, OrderedLastElementRand) {
    Test3([](int* list, int length, int i) {
        return getIthElementRand(list, length, i);
    });
}

// test case for the median element of an ordered array
void Test4(std::function<int(int*, int, int)> func) {
    int list[] = {1, 2, 3, 4, 5, 6, 7, 8, 9};
    int length = sizeof(list)/sizeof(int);
    int i = 4;
    const int result = func(list, length, i);
    EXPECT_EQ(result, 5);
}

TEST(RandomTest, OrderedMedianElement) {
    Test4([](int* list, int length, int i) {
        return getIthElement(list, length, i);
    });
}

TEST(RandomTest, OrderedMedianElementRand) {
    Test4([](int* list, int length, int i) {
        return getIthElementRand(list, length, i);
    });
}

// test case for the median element of an unordered array
void Test5(std::function<int(int*, int, int)> func) {
    int list[] = {4, 8, 9, 7, 3, 6, 5, 2, 1};
    int length = sizeof(list)/sizeof(int);
    int i = 4;
    const int result = func(list, length, i);
    EXPECT_EQ(result, 5);
}

TEST(RandomTest, UnorderedMedianElement) {
    Test5([](int* list, int length, int i) {
        return getIthElement(list, length, i);
    });
}

TEST(RandomTest, UnorderedMedianElementRand) {
    Test5([](int* list, int length, int i) {
        return getIthElementRand(list, length, i);
    });
}

// test generated with big inputs
void Test6(std::function<int(int*, int, int)> func) {
    int list[] = {58, 55, 5, 20, 11, 83, 16, 94, 12, 63, 83, 62, 62, 37, 55, 93, 83, 4, 89, 37, 57, 35, 21, 11, 66, 2, 90, 10, 58, 23, 85, 52, 12, 13, 8, 80, 1, 5, 89, 41, 10, 90, 85, 6, 21, 37, 9, 28, 66, 83, 86, 85, 32, 24, 43, 74, 44, 55, 20, 35, 54, 19, 71, 50, 26, 8, 2, 89, 78, 36, 1, 67, 55, 96, 40, 71, 22, 35, 50, 18, 89, 75, 90, 54, 6, 13, 43, 13, 67, 67, 13, 96, 10, 25, 30, 86, 9, 94, 47, 33};
    int length = sizeof(list)/sizeof(int);
    int i = 39;
    const int result = func(list, length, i);
    EXPECT_EQ(result, 33);
}

TEST(RandomTest, Array100) {
    Test6([](int* list, int length, int i) {
        return getIthElement(list, length, i);
    });
}

TEST(RandomTest, Array100Rand) {
    Test6([](int* list, int length, int i) {
        return getIthElementRand(list, length, i);
    });
}

// test for massive imputs
void Test7(std::function<int(int*, int, int)> func) {
    int list[] = {82, 69, 17, 60, 22, 28, 32, 12, 21, 62, 56, 51, 14, 33, 2, 81, 40, 67, 58, 19, 83, 31, 98, 58, 62, 21, 63, 75, 61, 40, 31, 91, 99, 15, 58, 61, 66, 96, 2, 74, 6, 93, 17, 61, 40, 68, 92, 93, 85, 16, 98, 68, 20, 56, 85, 8, 82, 35, 45, 31, 27, 37, 49, 17, 87, 14, 90, 70, 7, 16, 41, 17, 93, 33, 52, 45, 22, 38, 95, 17, 63, 78, 30, 36, 32, 4, 59, 74, 27, 7, 96, 64, 36, 82, 51, 39, 36, 4, 31, 38, 79, 11, 52, 29, 72, 26, 94, 91, 73, 71, 74, 30, 18, 65, 42, 88, 6, 99, 34, 35, 94, 42, 45, 33, 21, 75, 79, 58, 22, 49, 90, 78, 7, 15, 84, 5, 39, 50, 72, 76, 14, 8, 46, 32, 93, 43, 99, 74, 26, 18, 3, 23, 72, 86, 5, 57, 43, 86, 17, 26, 93, 83, 36, 88, 48, 36, 85, 65, 25, 95, 1, 46, 18, 16, 19, 25, 47, 21, 24, 16, 31, 32, 52, 77, 6, 50, 22, 76, 56, 96, 70, 80, 11, 58, 71, 27, 21, 81, 98, 78, 71, 76, 37, 52, 90, 91, 73, 77, 1, 5, 71, 21, 0, 84, 8, 38, 91, 53, 4, 6, 2, 97, 9, 94, 50, 45, 7, 0, 75, 32, 47, 14, 81, 56, 37, 15, 94, 98, 40, 46, 60, 49, 74, 68, 47, 58, 0, 3, 46, 1, 71, 81, 59, 42, 40, 16, 14, 56, 21, 73, 99, 69, 82, 49, 82, 32, 77, 25, 27, 99, 83, 38, 13, 98, 60, 35, 71, 22, 61, 55, 17, 84, 36, 0, 48, 14, 7, 32, 23, 90, 74, 9, 98, 59, 38, 42, 16, 44, 90, 76, 53, 33, 64, 91, 32, 37, 1, 15, 28, 56, 49, 49, 78, 21, 47, 74, 31, 56, 5, 69, 16, 7, 89, 19, 70, 69, 3, 55, 14, 83, 98, 22, 14, 87, 1, 94, 36, 61, 75, 32, 29, 57, 26, 40, 14, 24, 87, 65, 5, 82, 68, 49, 45, 56, 8, 96, 24, 26, 65, 3, 48, 26, 55, 1, 34, 40, 7, 7, 83, 37, 99, 13, 7, 30, 13, 41, 10, 16, 97, 2, 4, 52, 3, 44, 83, 17, 32, 23, 99, 7, 55, 9, 92, 6, 60, 2, 79, 96, 68, 80, 21, 79, 40, 54, 65, 13, 15, 38, 14, 29, 58, 3, 19, 45, 41, 44, 8, 26, 26, 41, 32, 75, 84, 87, 62, 75, 73, 27, 94, 56, 65, 47, 63, 76, 4, 13, 85, 23, 60, 60, 85, 53, 64, 76, 27, 32, 85, 1, 44, 42, 16, 60, 95, 40, 61, 36, 14, 50, 2, 92, 25, 48, 38, 4, 50, 93, 16, 63, 29, 56, 84, 31, 79, 60, 29, 3, 81, 72, 33, 39, 80, 63, 16, 23, 44, 77, 47, 15, 9, 48, 81, 33, 71, 63, 30, 0, 76, 75, 5, 61, 60, 45, 37, 41, 27, 57, 43, 75, 98, 41, 7, 3, 47, 42, 61, 3, 54, 9, 9, 91, 71, 95, 60, 61, 42, 14, 9, 7, 69, 79, 77, 70, 73, 6, 48, 57, 18, 89, 89, 13, 45, 24, 5, 53, 56, 60, 61, 33, 34, 55, 86, 52, 63, 41, 45, 51, 62, 6, 16, 1, 79, 75, 79, 7, 62, 31, 64, 41, 1, 85, 67, 56, 42, 12, 49, 60, 47, 56, 74, 96, 69, 12, 3, 16, 81, 68, 44, 19, 32, 96, 0, 12, 17, 68, 24, 91, 8, 78, 59, 31, 44, 99, 35, 28, 87, 90, 41, 39, 38, 72, 33, 37, 3, 73, 68, 78, 12, 39, 80, 28, 9, 86, 40, 1, 90, 12, 82, 98, 0, 15, 76, 70, 70, 55, 15, 44, 23, 64, 65, 0, 8, 85, 20, 80, 88, 5, 78, 69, 4, 56, 38, 63, 10, 98, 12, 42, 0, 45, 70, 40, 73, 19, 0, 58, 83, 72, 24, 40, 3, 22, 38, 65, 99, 35, 26, 69, 96, 0, 64, 4, 26, 25, 91, 88, 41, 16, 18, 16, 19, 67, 8, 17, 33, 43, 35, 6, 99, 60, 71, 96, 52, 29, 38, 89, 17, 44, 55, 80, 33, 67, 19, 40, 50, 36, 20, 86, 80, 76, 33, 55, 53, 46, 22, 14, 90, 89, 55, 21, 20, 1, 4, 36, 22, 85, 79, 98, 23, 71, 76, 19, 2, 75, 61, 13, 66, 39, 13, 47, 65, 32, 31, 40, 23, 45, 32, 61, 5, 35, 13, 82, 60, 95, 99, 56, 62, 95, 84, 26, 23, 22, 91, 90, 0, 30, 62, 74, 21, 32, 2, 39, 40, 23, 49, 96, 15, 72, 21, 81, 2, 61, 0, 27, 16, 88, 12, 14, 26, 51, 65, 24, 11, 53, 36, 33, 22, 53, 40, 71, 14, 68, 24, 29, 70, 5, 20, 4, 61, 40, 84, 23, 61, 93, 4, 80, 63, 53, 9, 11, 19, 82, 88, 28, 55, 93, 28, 4, 56, 67, 29, 56, 23, 13, 13, 52, 48, 57, 45, 73, 2, 16, 10, 99, 47, 17, 77, 76, 65, 75, 68, 69, 97, 71, 5, 54, 82, 71, 35, 8, 17, 24, 97, 78, 13, 87, 80, 15, 72, 72, 53, 97, 84, 39, 0, 96, 17, 89, 19, 97, 11, 24, 51, 74, 94, 7, 5, 32, 34, 11, 79, 62, 0, 43, 93, 4, 64, 18, 64, 21, 55, 13, 69, 79, 12, 10, 45, 15, 56, 65, 71, 80, 21, 20, 58, 30, 43, 40, 78, 40, 77, 68, 36, 3, 8, 80, 90, 54, 42, 92, 3, 14, 73, 62, 48, 6, 64, 65, 17, 58, 82, 63, 43, 64, 59, 56, 19, 38, 51, 56, 80, 77, 96, 27, 48, 31, 80, 67, 3, 73, 50, 27, 79, 62, 46, 0, 32, 28, 67, 25, 51, 57, 48, 3, 82, 60, 45, 37, 33, 75, 80, 20, 50, 47, 72, 95, 25, 69, 91, 48, 69, 6};
    int length = sizeof(list)/sizeof(int);
    int i = 346;
    const int result = func(list, length, i);
    EXPECT_EQ(result, 32);
}

TEST(RandomTest, Array1000) {
    Test7([](int* list, int length, int i) {
        return getIthElement(list, length, i);
    });
}

TEST(RandomTest, Array1000Rand) {
    Test7([](int* list, int length, int i) {
        return getIthElementRand(list, length, i);
    });
}

// test case for arrays with repeated elements
void Test8(std::function<int(int*, int, int)> func) {
    int list[] = {10,10,10,11};
    int length = sizeof(list)/sizeof(int);
    int i = 3;
    const int result = func(list, length, i);
    EXPECT_EQ(result, 11);
}

TEST(RandomTest, ArrayOfDoubles) {
    Test8([](int* list, int length, int i) {
        return getIthElement(list, length, i);
    });
}

TEST(RandomTest, ArrayOfDoublesRand) {
    Test8([](int* list, int length, int i) {
        return getIthElementRand(list, length, i);
    });
}

// another test for arrays with repeated elements
void Test9(std::function<int(int*, int, int)> func) {
    int list[] = {10,10,10,11};
    int length = sizeof(list)/sizeof(int);
    int i = 1;
    const int result = func(list, length, i);
    EXPECT_EQ(result, 10);
}

TEST(RandomTest, ArrayOfDoubles2) {
    Test9([](int* list, int length, int i) {
        return getIthElement(list, length, i);
    });
}

TEST(RandomTest, ArrayOfDoubles2Rand) {
    Test9([](int* list, int length, int i) {
        return getIthElementRand(list, length, i);
    });
}


int main(int argc, char **argv) {
    ::testing::InitGoogleTest(&argc, argv);
    return RUN_ALL_TESTS();
}

Writing test.cpp


In [5]:
# Compile and run the tests
!g++ -o test test.cpp ith_element.c -I /content/benchmark/googletest/googletest/include -L /content/benchmark/googletest/build/lib -lgtest -lpthread
!./test

[==========] Running 18 tests from 1 test suite.
[----------] Global test environment set-up.
[----------] 18 tests from RandomTest
[ RUN      ] RandomTest.NegativeInverseOrderedFirstElement
[       OK ] RandomTest.NegativeInverseOrderedFirstElement (0 ms)
[ RUN      ] RandomTest.NegativeInverseOrderedFirstElementRand
[       OK ] RandomTest.NegativeInverseOrderedFirstElementRand (0 ms)
[ RUN      ] RandomTest.OrderedFirstElement
[       OK ] RandomTest.OrderedFirstElement (0 ms)
[ RUN      ] RandomTest.OrderedFirstElementRand
[       OK ] RandomTest.OrderedFirstElementRand (0 ms)
[ RUN      ] RandomTest.OrderedLastElement
[       OK ] RandomTest.OrderedLastElement (0 ms)
[ RUN      ] RandomTest.OrderedLastElementRand
[       OK ] RandomTest.OrderedLastElementRand (0 ms)
[ RUN      ] RandomTest.OrderedMedianElement
[       OK ] RandomTest.OrderedMedianElement (0 ms)
[ RUN      ] RandomTest.OrderedMedianElementRand
[       OK ] RandomTest.OrderedMedianElementRand (0 ms)
[ RUN      ] Ran

## Benchmark

In [50]:
%%writefile benchmark.cpp

#include <benchmark/benchmark.h>
#include "ith_element.h"

#include <vector>
#include <random>

//generate a random vector with the specified size, and numbers from -5000 to 5000
std::vector<int> generate_random_array(int size) {
    std::vector<int> array(size);
    std::random_device rd;
    std::mt19937 gen(rd());
    std::uniform_int_distribution<> dis(-5000, 5000);

    for (int i = 0; i < size; ++i) {
        array[i] = dis(gen);  // Fill the array with random integers
    }

    return array;
}

// Benchmark random arrays given size and index
static void BM_RandomArrays(benchmark::State& state, int (*func)(int*, int, int)) {
    std::vector<int> array = generate_random_array(state.range(0));

    for (auto _ : state) {
        if (state.range(1) >= state.range(0)) continue; //skip if index is out-of-bound
        std::vector<int> array_copy = array;
        int* array_ptr = array_copy.data(); //copy the array to reuse the same at each iteration
        benchmark::DoNotOptimize(array_ptr);  // Avoid compiler optimizations

        int result = func(array_ptr, array_copy.size(), state.range(1));  // Call the function
        benchmark::DoNotOptimize(result);
        benchmark::ClobberMemory();  // Prevents compiler from reordering operations
    }
    state.SetComplexityN(state.range(0)); //set the complexity based on the size of the array
}

// Benchmark of an ordered array given an index
static void BM_OrderedArray(benchmark::State& state, int (*func)(int*, int, int)) {
  std::vector<int> array;
  //generate an ordered array of 100 elements
  for (int i = 1; i <= 1000000; ++i) {
      array.push_back(i);
  }

  for (auto _ : state){
    std::vector<int> array_copy = array;
    int* array_ptr = array_copy.data();
    benchmark::DoNotOptimize(array_ptr);

    int result = getIthElement(array_ptr, array_copy.size(), state.range(0));
    benchmark::DoNotOptimize(result);
    benchmark::ClobberMemory();
  }
  state.SetComplexityN(array.size());
}

static void BM_MedianOfMedians(benchmark::State& state) {
  BM_RandomArrays(state, getIthElement);
}

static void BM_OrderedMedianOfMedians(benchmark::State& state) {
  BM_OrderedArray(state, getIthElement);
}

static void BM_QuickSelect(benchmark::State& state) {
  BM_RandomArrays(state, getIthElementRand);
}

static void BM_OrderedQuickSelect(benchmark::State& state) {
  BM_OrderedArray(state, getIthElementRand);
}

// Register the function as a benchmark and define inputs
//  Gives size, index
BENCHMARK(BM_MedianOfMedians)

  ->Args({4096, 2048})
  ->Args({4096, 3658})
  ->Args({8192, 0})
  ->Args({8192, 4096})
  ->Args({8192, 6000})
  ->Args({8192, 8191})
  ->Args({81920, 6000})
  ->Args({81920000, 600320})
  ->Args({8192000, 60})

  ->Complexity(benchmark::oN); //Ensures benchmark interprets the results as linear, and it will give an RMS error based on how well the actual timings match linear growth.

// index
BENCHMARK(BM_OrderedMedianOfMedians)
  ->Args({0})
  ->Args({5})
  ->Args({9})
  ->Args({20})
  ->Args({50})
  ->Args({76})
  ->Args({99})

  ->Complexity(benchmark::oN);

BENCHMARK(BM_QuickSelect)

  ->Args({4096, 2048})
  ->Args({4096, 3658})
  ->Args({8192, 0})
  ->Args({8192, 4096})
  ->Args({8192, 8191})
  ->Args({81920, 6000})
  ->Args({81920000, 600320})
  ->Args({8192000, 60})

  ->Complexity(benchmark::oN);

// index
BENCHMARK(BM_OrderedQuickSelect)
  ->Args({0})
  ->Args({5})
  ->Args({9})
  ->Args({20})
  ->Args({50})
  ->Args({76})
  ->Args({99})

  ->Complexity(benchmark::oN);

BENCHMARK_MAIN();

Overwriting benchmark.cpp


In [51]:
# Compile and run the benchmark
!g++ -o bench benchmark.cpp ith_element.c -lbenchmark -lpthread
!./bench

2024-10-22T13:20:10+00:00
Running ./bench
Run on (2 X 2200.21 MHz CPU s)
CPU Caches:
  L1 Data 32 KiB (x1)
  L1 Instruction 32 KiB (x1)
  L2 Unified 256 KiB (x1)
  L3 Unified 56320 KiB (x1)
Load Average: 0.81, 0.87, 0.79
-----------------------------------------------------------------------------
Benchmark                                   Time             CPU   Iterations
-----------------------------------------------------------------------------
BM_MedianOfMedians/4096/2048           474442 ns       467446 ns         1702
BM_MedianOfMedians/4096/3658           563618 ns       523939 ns         1307
BM_MedianOfMedians/8192/0             1086396 ns      1033366 ns          652
BM_MedianOfMedians/8192/4096          1056743 ns      1030221 ns          645
BM_MedianOfMedians/8192/6000           863706 ns       853511 ns          631
BM_MedianOfMedians/8192/8191           851061 ns       842722 ns          848
BM_MedianOfMedians/81920/6000         8304015 ns      8285379 ns           85